In [130]:
import pandas as pd
import numpy as np    
from sklearn.model_selection import train_test_split     
from sklearn import metrics 
from sklearn.preprocessing import OneHotEncoder   
from sklearn.preprocessing import StandardScaler    
from sklearn.preprocessing import LabelEncoder  
from sklearn.metrics import f1_score  
from sklearn.pipeline import Pipeline  
from sklearn.impute import SimpleImputer  
from sklearn.compose import ColumnTransformer
import multiprocessing
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier 
import matplotlib.pyplot as plt



In [2]:
trc=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Train\TRAIN_CATEGORICAL_METADATA.xlsx')  
trq=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Train\TRAIN_QUANTITATIVE_METADATA.xlsx')    
trf=pd.read_csv(r'C:\Users\Family\Desktop\WIDS-Com\Train\TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv')
trs=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Train\TRAINING_SOLUTIONS.xlsx')  
tsc=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Test\TEST_CATEGORICAL.xlsx')
tsq=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Train\TEST_QUANTITATIVE_METADATA.xlsx')    
tsf=pd.read_csv(r'C:\Users\Family\Desktop\WIDS-Com\Test\TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')    
sub=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Submission\SAMPLE_SUBMISSION.xlsx')    
dic=pd.read_excel(r'C:\Users\Family\Desktop\WIDS-Com\Submission\Data Dictionary.xlsx')

Data Merging 

In [3]:
cq = pd.merge(trc, trq, on='participant_id', how='left')
feat = pd.merge(cq, trf, on='participant_id', how='left')  
qc = pd.merge(tsc, tsq, on='participant_id', how='left')
train = pd.merge(feat, trs, on='participant_id', how='left') 
test = pd.merge(qc, tsf, on='participant_id', how='left')  

Data Preprocessing

In [4]:
train.describe()

,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ,EHQ_EHQ_Total,...,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn,ADHD_Outcome,Sex_F
count,1213.000000,1213.000000,1202.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,...,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000
mean,2017.652102,2.014839,0.424293,2.080791,2.288541,17.641385,24.892828,14.122012,24.719703,58.876917,...,-0.004378,0.001610,0.115171,0.049984,0.058144,0.093527,0.089403,0.128946,0.685078,0.342951
std,1.122522,1.135147,0.687470,3.164636,0.758348,4.004639,17.025899,7.201023,17.171113,49.855501,...,0.042900,0.047424,0.057128,0.051664,0.057674,0.054594,0.058036,0.058490,0.464676,0.474891
min,2015.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-100.000000,...,-0.178688,-0.138048,-0.070094,-0.153540,-0.131455,-0.085566,-0.204160,-0.083077,0.000000,0.000000
25%,2017.000000,1.000000,0.000000,0.000000,2.000000,15.000000,0.000000,12.000000,5.000000,45.600000,...,-0.033798,-0.030132,0.080291,0.015827,0.022316,0.059621,0.053224,0.090459,0.000000,0.000000
50%,2018.000000,1.000000,0.000000,0.000000,2.000000,18.000000,30.000000,18.000000,30.000000,76.670000,...,-0.003724,0.000990,0.113640,0.052705,0.059151,0.093397,0.088612,0.127913,1.000000,0.000000
75%,2019.000000,3.000000,1.000000,2.000000,3.000000,21.000000,40.000000,21.000000,40.000000,93.400000,...,0.024007,0.032268,0.150524,0.082526,0.095192,0.127144,0.127613,0.166523,1.000000,1.000000
max,2020.000000,4.000000,3.000000,11.000000,4.000000,21.000000,45.000000,21.000000,45.000000,100.000000,...,0.180562,0.192015,0.375635,0.228748,0.322084,0.348153,0.267162,0.414304,1.000000,1.000000


In [132]:
unimportant_features = important_features[important_features["Importance"] == 0.000000]
important_features = important_features.drop(unimportant_features.index)["Feature"].tolist()

In [134]:
c_train = train.copy()
c_test = test.copy()    
unimportant_features_list = unimportant_features['Feature'].tolist()
c_train = c_train.drop(unimportant_features_list, axis=1)
c_test = c_test.drop(unimportant_features_list, axis=1)  

In [102]:
X = c_train.drop(['participant_id', 'ADHD_Outcome', 'Sex_F'], axis=1, errors='ignore')
y_adhd = c_train['ADHD_Outcome']
y_sex = c_train['Sex_F']

In [104]:
y_adhd

0       1
1       1
2       1
3       1
4       1
       ..
1208    0
1209    0
1210    0
1211    0
1212    0
Name: ADHD_Outcome, Length: 1213, dtype: int64

In [103]:
X

,Barratt_Barratt_P2_Occ,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,...,191throw_198thcolumn,192throw_193thcolumn,192throw_194thcolumn,192throw_198thcolumn,193throw_197thcolumn,193throw_198thcolumn,194throw_195thcolumn,194throw_198thcolumn,195throw_197thcolumn,197throw_199thcolumn
0,45,0,6,1,5,0,5,1,0,10,...,-0.046135,0.016116,0.008075,0.091747,0.013313,0.078021,0.106554,-0.005394,-0.072599,0.093230
1,0,0,18,6,8,7,8,10,4,5,...,-0.019675,0.005572,0.036550,0.024132,0.028336,-0.014337,0.064698,0.028457,-0.001084,0.064094
2,0,1,14,2,8,5,7,6,4,9,...,-0.100913,0.021315,-0.116377,0.115415,-0.013479,-0.017301,0.007620,-0.037527,0.018640,-0.009425
3,0,6,24,4,16,9,10,8,4,6,...,-0.031084,0.039091,-0.039664,0.075565,0.060234,-0.017723,0.080065,-0.021116,0.025618,0.065336
4,0,1,18,4,11,4,10,7,3,9,...,0.017720,0.032003,0.021723,0.066198,0.006562,0.039195,0.067613,-0.033654,-0.031508,0.112522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,40,1,9,2,7,3,6,2,0,9,...,0.097734,-0.023523,-0.052904,0.064999,-0.003852,0.004950,0.050474,0.020667,-0.032095,0.159979
1209,0,6,18,3,11,9,5,7,4,7,...,0.000034,-0.038774,-0.101987,0.015559,0.053053,0.063333,-0.000737,0.021207,-0.028994,0.084720
1210,0,3,4,1,3,0,0,1,0,10,...,0.037547,-0.051468,-0.028413,0.053167,0.040222,0.061938,-0.041679,0.063551,0.079623,0.010093
1211,45,4,9,0,9,3,5,0,0,9,...,-0.012576,-0.077030,0.025237,0.100404,-0.053356,-0.003538,0.062659,-0.027684,0.065225,0.106790


In [117]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(exclude=['object']).columns.tolist()
#('imputer', SimpleImputer(strategy='median')),
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

#('imputer', SimpleImputer(strategy='most_frequent'))
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [118]:
X_tr = preprocessor.fit_transform(X)
X_ts = preprocessor.transform(test.drop('participant_id', axis=1))

In [121]:
y_tr = pd.merge(y_adhd , y_sex, left_index=True, right_index=True)   

In [68]:
y_sex.head

<bound method NDFrame.head of 0       1
1       0
2       0
3       1
4       1
       ..
1208    0
1209    1
1210    1
1211    0
1212    0
Name: Sex_F, Length: 1213, dtype: int64>

In [10]:
y_tr.head()

,ADHD_Outcome,Sex_F
0,1,1
1,1,0
2,1,0
3,1,1
4,1,1


feature engineering

In [115]:
X["emotional_affectness"] = X["SDQ_SDQ_Emotional_Problems"] * X["SDQ_SDQ_Internalizing"]
X["physical_affectness"] = X["SDQ_SDQ_Externalizing"] * X["SDQ_SDQ_Hyperactivity"]


In [116]:
test["emotional_affectness"] = test["SDQ_SDQ_Emotional_Problems"] * test["SDQ_SDQ_Internalizing"]
test["physical_affectness"] = test["SDQ_SDQ_Externalizing"] * test["SDQ_SDQ_Hyperactivity"]

In [70]:
X

,Barratt_Barratt_P2_Occ,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,...,192throw_193thcolumn,192throw_194thcolumn,192throw_198thcolumn,193throw_197thcolumn,193throw_198thcolumn,194throw_195thcolumn,194throw_198thcolumn,195throw_197thcolumn,197throw_199thcolumn,emotional_affectness
0,45,0,6,1,5,0,5,1,0,10,...,0.016116,0.008075,0.091747,0.013313,0.078021,0.106554,-0.005394,-0.072599,0.093230,1
1,0,0,18,6,8,7,8,10,4,5,...,0.005572,0.036550,0.024132,0.028336,-0.014337,0.064698,0.028457,-0.001084,0.064094,60
2,0,1,14,2,8,5,7,6,4,9,...,0.021315,-0.116377,0.115415,-0.013479,-0.017301,0.007620,-0.037527,0.018640,-0.009425,12
3,0,6,24,4,16,9,10,8,4,6,...,0.039091,-0.039664,0.075565,0.060234,-0.017723,0.080065,-0.021116,0.025618,0.065336,32
4,0,1,18,4,11,4,10,7,3,9,...,0.032003,0.021723,0.066198,0.006562,0.039195,0.067613,-0.033654,-0.031508,0.112522,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,40,1,9,2,7,3,6,2,0,9,...,-0.023523,-0.052904,0.064999,-0.003852,0.004950,0.050474,0.020667,-0.032095,0.159979,4
1209,0,6,18,3,11,9,5,7,4,7,...,-0.038774,-0.101987,0.015559,0.053053,0.063333,-0.000737,0.021207,-0.028994,0.084720,21
1210,0,3,4,1,3,0,0,1,0,10,...,-0.051468,-0.028413,0.053167,0.040222,0.061938,-0.041679,0.063551,0.079623,0.010093,1
1211,45,4,9,0,9,3,5,0,0,9,...,-0.077030,0.025237,0.100404,-0.053356,-0.003538,0.062659,-0.027684,0.065225,0.106790,0


Model Training

xgboost

In [139]:
X_tr, X_val, y_tr, y_val = train_test_split(X_tr, y_tr, test_size=0.2, random_state=42)


In [154]:
X_model = XGBClassifier(max_depth= 6 ,n_estimators=1000, learning_rate=0.05, n_jobs=multiprocessing.cpu_count())  

ad_m = X_model.fit(X_tr, y_adhd) 

XGBoostError: [01:21:37] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\data\data.cc:508: Check failed: this->labels.Size() % this->num_row_ == 0 (93 vs. 0) : Incorrect size for labels.

In [ ]:
y_val_pred = ad_m.predict(X_val)

f1 = f1_score(y_val['ADHD_Outcome'], y_val_pred, average='macro')
print(f"Validation F1-score: {f1:.4f}")

Validation F1-score: 1.0000


0.7056

In [91]:
adhd_pred = ad_m.predict(X_ts)   

In [151]:

X_model = XGBClassifier(max_depth=3)

X_model.fit(X_tr, y_tr['Sex_F'])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [153]:
y_val_pred_x = X_model.predict(X_val)

f1_sex = f1_score(y_val['Sex_F'], y_val_pred_x, average='macro')
print(f"Validation F1-score: {f1_sex:.4f}")

Validation F1-score: 0.4109


In [93]:
sex_pred = X_model.predict(X_ts)   

xgboost with multioutput : Validation F1-score: 0.5189


feature_importances

In [131]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_tr, y_tr)

X_tr_df = pd.DataFrame(X_tr, columns=numerical_features + categorical_features)

feature_importance = model.feature_importances_
important_features = pd.DataFrame({"Feature": X_tr_df.columns, "Importance": feature_importance})
important_features.sort_values("Importance", ascending=False)


,Feature,Importance
7417,physical_affectness,0.007017
4,SDQ_SDQ_Externalizing,0.005077
2,SDQ_SDQ_Difficulties_Total,0.004147
5,SDQ_SDQ_Generating_Impact,0.004066
6,SDQ_SDQ_Hyperactivity,0.003773
...,...,...
3652,57throw_71thcolumn,0.000000
3653,57throw_76thcolumn,0.000000
3657,57throw_85thcolumn,0.000000
3658,57throw_93thcolumn,0.000000


LGBM

In [157]:
base_model = LGBMClassifier(max_depth= 3)

base_model.fit(X_tr, y_tr['ADHD_Outcome'])


[LightGBM] [Info] Number of positive: 117, number of negative: 43
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024336 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 407528
[LightGBM] [Info] Number of data points in the train set: 160, number of used features: 7418
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.731250 -> initscore=1.000974
[LightGBM] [Info] Start training from score 1.000974
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

LGBMClassifier(max_depth=3)

Validation ( makes you feel special with good scores then embarrasses you in the public leaderboard😤)

In [162]:
X_tr, X_val, y_tr, y_val = train_test_split(X_tr, y_tr, test_size=0.2, random_state=42)

y_val_pred = base_model.predict(X_val)

f1_lgbm = f1_score(y_val['ADHD_Outcome'], y_val_pred, average='macro')
print(f"Validation F1-score: {f1_lgbm:.4f}")



Validation F1-score: 1.0000


c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [167]:
lgbm_model = LGBMClassifier(max_depth= 3)

lgbm_model = lgbm_model.fit(X_tr, y_tr['Sex_F'])    


[LightGBM] [Info] Number of positive: 21, number of negative: 43
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008392 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 170484
[LightGBM] [Info] Number of data points in the train set: 64, number of used features: 7418
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.328125 -> initscore=-0.716678
[LightGBM] [Info] Start training from score -0.716678
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [171]:
y_val_pred = lgbm_model.predict(X_val)

f1_lgb = f1_score(y_val['Sex_F'], y_val_pred, average='macro')
print(f"Validation F1-score: {f1_lgb:.4f}")



Validation F1-score: 1.0000


c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [173]:
adhd_lgbm = base_model.predict(X_ts)    
sex_lgbm = lgbm_model.predict(X_ts) 

c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeClassifier

model = MultiOutputClassifier(make_pipeline(
    ColumnTransformer([('imputer', SimpleImputer(), X_tr)],
                     remainder='passthrough',
                     verbose_feature_names_out=False),
    
    ColumnTransformer([('log', 
                       FunctionTransformer(np.log1p), log_features)],
                     remainder='passthrough'),
    
    MinMaxScaler(),
    
    RidgeClassifier(alpha=100)
))

model.fit(pd.DataFrame(X_tr, columns=features), y_tr)


MultiOutputClassifier(estimator=Pipeline(steps=[('columntransformer-1',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   transformers=[('imputer',
                                                                                  SimpleImputer(),
                                                                                  Index(['Basic_Demos_Enroll_Year', 'Basic_Demos_Study_Site',
       'PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race',
       'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu',
       'Barratt_Barratt_P1_Oc...
       '196throw_199thcolumn', '197throw_198thcolumn', '197throw_199thcolumn',
       '198throw_199thcolumn'],
      dtype='object', length=19927))],
                                                                   verbose_feature_names_out=False)),
                                                ('columntransformer-2',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   transformers=[('log',
                                                                                  FunctionTransformer(func=<ufunc 'log1p'>),
                                                                                  [])])),
                                                ('minmaxscaler',
                                                 MinMaxScaler()),
                                                ('ridgeclassifier',
                                                 RidgeClassifier(alpha=100))]))

In [21]:
X_tr_df = pd.DataFrame(X_tr, columns=numerical_features + categorical_features)
X_val_df = pd.DataFrame(X_val, columns=numerical_features + categorical_features)

X_tr, X_val, y_tr, y_val = train_test_split(X_tr_df, y_tr, test_size=0.2, random_state=42)

model.fit(X_tr, y_tr)

y_val_pred = model.predict(X_val)

In [23]:
print('f1: ', f1_score(y_val_pred, y_val, average='micro'))


f1:  0.6640926640926641


In [ ]:
features = X_tr.columns.tolist()  

In [22]:
import numpy as np
import pandas as pd
import multiprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.linear_model import RidgeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier

# Define models
ridge_model = MultiOutputClassifier(make_pipeline(
    ColumnTransformer([('imputer', SimpleImputer(), features)],
                     remainder='passthrough',
                     verbose_feature_names_out=False),
    
    ColumnTransformer([('log', FunctionTransformer(np.log1p), log_features)],
                     remainder='passthrough'),
    
    MinMaxScaler(),
    
    RidgeClassifier(alpha=100)
))

xgb_model = XGBClassifier(
    max_depth=3,
    n_estimators=1000,
    learning_rate=0.05,
    n_jobs=multiprocessing.cpu_count()
)

# Stacking model
stacked_model = MultiOutputClassifier(StackingClassifier(
    estimators=[('ridge', ridge_model), ('xgb', xgb_model)],
    final_estimator=RidgeClassifier(),  # Meta-classifier
    passthrough=True,  # Use original features in meta-model
    n_jobs=-1
))

# Train the stacked model
stacked_model.fit(pd.DataFrame(X_tr, columns=features), y_tr)


NameError: name 'log_features' is not defined

Predicting the Hedin target


In [ ]:
#lgbm

y_pred = multi_target_model.predict(X_ts)


NameError: name 'multi_target_model' is not defined

In [32]:
y_pred_rg = model.predict(X_ts_df)


Submission

In [176]:
test_pred = test[['participant_id']].copy()
test_pred['ADHD_Outcome'] = adhd_lgbm
test_pred['Sex_F'] = sex_lgbm

test_pred.to_csv(f"now_submission_{f1_lgb:.4f}.csv", index=False)
